# Challenge 1

The heart disease dataset is a classic dataset that contains various health metrics (age, sex, chest pain type, blood pressure, cholesterol, etc.) related to diagnosing heart disease (binary classification: presence or absence of heart disease).

In [71]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the dataset (change the path if needed)
df = pd.read_csv('../data/heart.csv')

In [72]:
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


We are going to try to predict the presence of hart disease suing this features, starting with a classical baseline method and trying to improve on that result with a series of ensembled approaches.

In [73]:
X = df.drop(columns="target")
y = df["target"]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

# Feature scaling (for certain models, e.g., SVM or logistic regression, not always necessary for trees)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Baseline model : decision Tree

We'll train a decision tree as our baseline model and evaluate it using accuracy.

In [74]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report

# Train Decision Tree
model = DecisionTreeClassifier(max_depth=1)
model.fit(X_train_scaled, y_train)

# Predictions
y_pred_train = model.predict(X_train_scaled)
y_pred_test = model.predict(X_test_scaled)

# Accuracy
print(f"Train Accuracy: {accuracy_score(y_train, y_pred_train):.4f}")
print(f"Test Accuracy: {accuracy_score(y_test, y_pred_test):.4f}")

# Detailed classification metrics
print(classification_report(y_test, y_pred_test))


Train Accuracy: 0.7577
Test Accuracy: 0.7632
              precision    recall  f1-score   support

           0       0.73      0.73      0.73        33
           1       0.79      0.79      0.79        43

    accuracy                           0.76        76
   macro avg       0.76      0.76      0.76        76
weighted avg       0.76      0.76      0.76        76



We can see that this model is overfitting. This is expected, decision trees, especially deep ones  are notorious agressive at exploiting the data available. But that also makes them highly variant: a small change on the tree/data makes for potentially large changes in performance.

In [75]:
# Run the same code again a couple of times. 
# You can see that the Train Accuracy is always 100% (overfitting) and the Test Accuracy is all over the place. 
# This is undesirable: our method is not generalizing and has high variance

# Bagging: reducing variance

Bagging improves models because it reduces variance by averaging the predictions of multiple models trained on different subsets of the training data. This averaging effect reduces the sensitivity of the overall model to any one dataset or model, making the final prediction more stable and less prone to overfitting.

- High-variance models, like decision trees, tend to overfit the training data. This means that small changes in the training data can lead to large changes in the model’s predictions. For example, a decision tree trained on one subset of data might look completely different from a decision tree trained on another subset. This leads to high variance, where the model’s performance fluctuates a lot depending on the specific data it was trained on.
- Once all the individual models are trained, Bagging combines their predictions by averaging them (for regression) or using a majority vote (for classification). The key idea here is that the errors in each individual model are somewhat independent because they are trained on different bootstrap samples. Some models will make errors in one direction, while others might make errors in another. When you average these predictions, the errors cancel out, reducing the overall variability (variance) of the final model.

In [76]:
# Create and Train a BaggingClassifier. 
# Use as base estimator a weak decision tree (max_depth=1) and 100 estimators to really over a lot of different data samples
# Print the train and test accuracy

from sklearn.ensemble import BaggingClassifier

bagging_reg = BaggingClassifier(
    DecisionTreeClassifier(max_depth=1), # depth 1 to force tree to be "weak"
    n_estimators=10, # 10 trees
    max_samples=100, # we limit each weaker tree to 100 datapoints
    random_state=1) # fixing random state because I want my examples to work and to look smart

bagging_reg.fit(X_train_scaled, y_train)
# Accuracy
print(f"Train Accuracy: {bagging_reg.score(X_train_scaled, y_train)}")
print(f"Test Accuracy: {bagging_reg.score(X_test_scaled,y_test)}")

# Detailed classification metrics
print(classification_report(y_test, y_pred_test))

Train Accuracy: 0.7577092511013216
Test Accuracy: 0.7631578947368421
              precision    recall  f1-score   support

           0       0.73      0.73      0.73        33
           1       0.79      0.79      0.79        43

    accuracy                           0.76        76
   macro avg       0.76      0.76      0.76        76
weighted avg       0.76      0.76      0.76        76



You can probably see a modest improvement in score, but most importantly, the overfitting is mostly gone. This is because averaging over multiple datasets stabilizes the high variance of the base model. 

In [77]:
# Run the same code again a couple of times. 
# You can see that consistently the Train Accuracy is close to the Test Accuracy. 

# Boosting: reducing bias

Now we’ll apply AdaBoost with decision trees as weak learners. This will sequentially improve the model by focusing on difficult cases.

Boosting reduces bias by sequentially training a series of weak learners (often simple models like decision trees) where each subsequent model focuses on the mistakes made by the previous models. The key idea behind boosting is to incrementally improve the model by correcting errors, which helps to reduce bias, especially when the initial model is too simple and underfits the data.

- Boosting typically uses weak learners, which are models that perform only slightly better than random guessing. For example, in classification, a weak learner might be a shallow decision tree (a "stump") with just a few levels. Weak learners usually have high bias, meaning they are too simplistic and don't capture the underlying patterns in the data well. As a result, they underfit the data.

- In each iteration, boosting trains a new model that tries to correct the errors made by the earlier models. If an instance was misclassified by the first weak learner, it will receive a higher weight, so the next model pays more attention to it. As the sequence of models progresses, the ensemble collectively focuses more on the difficult-to-predict instances. Over time, the combined models become better at fitting the data, as they successively reduce the bias (systematic error) by adjusting for earlier mistakes.

In [78]:
# Create and Train a AdaBoostClassifier. 
# Use as base estimator a weak decision tree (max_depth=1) and 100 estimators to really target the specific behaviors of this phenomenon
# Print the train and test accuracy

from sklearn.ensemble import AdaBoostClassifier

ada_reg = AdaBoostClassifier(DecisionTreeClassifier(max_depth=1), # you can overfit a bit because you compensate afterwards
                            n_estimators=10, # same 10 trees. You usually use faaaar more estimators
                            random_state=1 # once a coward, always a coward
                            )
ada_reg.fit(X_train_scaled, y_train)

# Accuracy
print(f"Train Accuracy: {ada_reg.score(X_train_scaled, y_train)}")
print(f"Test Accuracy: {ada_reg.score(X_test_scaled,y_test)}")

# Detailed classification metrics
print(classification_report(y_test, y_pred_test))


Train Accuracy: 0.8458149779735683
Test Accuracy: 0.868421052631579
              precision    recall  f1-score   support

           0       0.73      0.73      0.73        33
           1       0.79      0.79      0.79        43

    accuracy                           0.76        76
   macro avg       0.76      0.76      0.76        76
weighted avg       0.76      0.76      0.76        76



You can probably see a good improvement in score, but overfitting rearing it's ugly head a gain (not as much as in the base model). This is because the iterative correction of adaboost really allows the model to focus on the specifics of this problem, at a cost of overexploiting the dataset.

In [79]:
# Run the same code again a couple of times. 
# You can see that the test Accuracy will mostly be pretty good, even if some times it get's lower or higher scores (high variance, low bias)
# You can see also that consistently the Train Accuracy is higher than the Test Accuracy,indicating some (not extreme) overfitting 